# Predicting Monthly Microbusiness Density in a given area

## 1. Problem Definition
Predict monthly microbusiness density n a given area to aid policymakers gain visibility into microbusiness

## 2. Data
Data is gotten from: https://www.kaggle.com/competitions/godaddy-microbusiness-density-forecasting/data

## 3. Evaluation
Submissions are evaluated on `SMAPE` between forecasts and actual values. We define `SMAPE` = 0 when the actual and predicted values are both 0.

## 4. Features
**Training data**
* `row_id` - An ID code for the row.
* `cfips` - A unique identifier for each county using the Federal Information Processing System. The first two digits correspond to the state FIPS code, while the following 3 represent the county.
* `county_name` - The written name of the county.
* `state_name` - The name of the state.
* `first_day_of_month` - The date of the first day of the month.
* `microbusiness_density` - Microbusinesses per 100 people over the age of 18 in the given county. This is the target variable. The population figures used to calculate the density are on a two-year lag due to the pace of update provided by the U.S. Census Bureau, which provides the underlying population data annually. 2021 density figures are calculated using 2019 population figures, etc.
* `active` - The raw count of microbusinesses in the county. Not provided for the test set.

**Test data**
* `row_id` - An ID code for the row.
* `cfips` - A unique identifier for each county using the Federal Information Processing System. The first two digits correspond to the state FIPS code, while the following 3 represent the county.
* `first_day_of_month` - The date of the first day of the month.

In [139]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [140]:
df_microbusiness = pd.read_csv("C:/Users\Boma Allwell-Brown/Machine learning/GoDaddy-Microbusiness-Density-Forecasting/godaddy-microbusiness-density-forecasting/train.csv", 
                               parse_dates=["first_day_of_month"])

In [141]:
df_microbusiness.head(10)

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243
5,1001_2020-01-01,1001,Autauga County,Alabama,2020-01-01,2.969090,1242
6,1001_2020-02-01,1001,Autauga County,Alabama,2020-02-01,2.909326,1217
7,1001_2020-03-01,1001,Autauga County,Alabama,2020-03-01,2.933231,1227
8,1001_2020-04-01,1001,Autauga County,Alabama,2020-04-01,3.000167,1255
9,1001_2020-05-01,1001,Autauga County,Alabama,2020-05-01,3.004948,1257


In [142]:
# Active column is not required for model therefore should be dropped
df_microbusiness.drop(columns='active', inplace=True)
df_microbusiness

,row_id,cfips,county,state,first_day_of_month,microbusiness_density
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233
...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395


In [143]:
df_microbusiness

,row_id,cfips,county,state,first_day_of_month,microbusiness_density
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233
...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395


In [144]:
from sklearn.linear_model import SGDRegressor

In [145]:
df_microbusiness.head(20)

,row_id,cfips,county,state,first_day_of_month,microbusiness_density
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233
5,1001_2020-01-01,1001,Autauga County,Alabama,2020-01-01,2.969090
6,1001_2020-02-01,1001,Autauga County,Alabama,2020-02-01,2.909326
7,1001_2020-03-01,1001,Autauga County,Alabama,2020-03-01,2.933231
8,1001_2020-04-01,1001,Autauga County,Alabama,2020-04-01,3.000167
9,1001_2020-05-01,1001,Autauga County,Alabama,2020-05-01,3.004948


In [146]:
df_temp = df_microbusiness.copy()

In [147]:
df_temp

,row_id,cfips,county,state,first_day_of_month,microbusiness_density
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233
...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395


In [148]:
# Create a column for month and year
df_temp['month'] = df_temp.first_day_of_month.dt.month
df_temp['year'] = df_temp.first_day_of_month.dt.year

In [149]:
df_temp

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,month,year
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,8,2019
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,9,2019
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,10,2019
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,11,2019
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,12,2019
...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,6,2022
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,7,2022
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,8,2022
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,9,2022


In [150]:
# To identify columns with string
for label, content in df_temp.items():
    if pd.api.types.is_string_dtype(content):
        print(label)

row_id
county
state


In [151]:
# Convert string columns to category
for label, content in df_temp.items():
    if pd.api.types.is_string_dtype(content):
        df_temp[label] = content.astype('category').cat.as_ordered()

In [152]:
df_temp.head(30)

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,month,year
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,8,2019
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,9,2019
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,10,2019
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,11,2019
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,12,2019
5,1001_2020-01-01,1001,Autauga County,Alabama,2020-01-01,2.969090,1,2020
6,1001_2020-02-01,1001,Autauga County,Alabama,2020-02-01,2.909326,2,2020
7,1001_2020-03-01,1001,Autauga County,Alabama,2020-03-01,2.933231,3,2020
8,1001_2020-04-01,1001,Autauga County,Alabama,2020-04-01,3.000167,4,2020
9,1001_2020-05-01,1001,Autauga County,Alabama,2020-05-01,3.004948,5,2020


In [153]:
df_temp.dtypes

row_id                         category
cfips                             int64
county                         category
state                          category
first_day_of_month       datetime64[ns]
microbusiness_density           float64
month                             int64
year                              int64
dtype: object

In [154]:
df_temp.to_csv('microbusiness_density_forecasting_temp.csv')

In [155]:
# Check for Null values
df_temp.isna().sum()

row_id                   0
cfips                    0
county                   0
state                    0
first_day_of_month       0
microbusiness_density    0
month                    0
year                     0
dtype: int64

In [156]:
df_temp.sort_values('first_day_of_month', inplace=True)
df_temp

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,month,year
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,8,2019
41613,21153_2019-08-01,21153,Magoffin County,Kentucky,2019-08-01,0.336289,8,2019
80652,39057_2019-08-01,39057,Greene County,Ohio,2019-08-01,5.733927,8,2019
5460,5063_2019-08-01,5063,Independence County,Arkansas,2019-08-01,1.865025,8,2019
116805,54021_2019-08-01,54021,Gilmer County,West Virginia,2019-08-01,0.526540,8,2019
...,...,...,...,...,...,...,...,...
26675,17181_2022-10-01,17181,Union County,Illinois,2022-10-01,1.637489,10,2022
95588,47053_2022-10-01,47053,Gibson County,Tennessee,2022-10-01,2.517291,10,2022
43133,21229_2022-10-01,21229,Washington County,Kentucky,2022-10-01,1.592288,10,2022
105143,48353_2022-10-01,48353,Nolan County,Texas,2022-10-01,1.502534,10,2022


In [157]:
df_temp['row_id'].dtype

CategoricalDtype(categories=['10001_2019-08-01', '10001_2019-09-01', '10001_2019-10-01',
                  '10001_2019-11-01', '10001_2019-12-01', '10001_2020-01-01',
                  '10001_2020-02-01', '10001_2020-03-01', '10001_2020-04-01',
                  '10001_2020-05-01',
                  ...
                  '9015_2022-01-01', '9015_2022-02-01', '9015_2022-03-01',
                  '9015_2022-04-01', '9015_2022-05-01', '9015_2022-06-01',
                  '9015_2022-07-01', '9015_2022-08-01', '9015_2022-09-01',
                  '9015_2022-10-01'],
, ordered=True)

In [158]:
df_temp['row_id'].dtype

CategoricalDtype(categories=['10001_2019-08-01', '10001_2019-09-01', '10001_2019-10-01',
                  '10001_2019-11-01', '10001_2019-12-01', '10001_2020-01-01',
                  '10001_2020-02-01', '10001_2020-03-01', '10001_2020-04-01',
                  '10001_2020-05-01',
                  ...
                  '9015_2022-01-01', '9015_2022-02-01', '9015_2022-03-01',
                  '9015_2022-04-01', '9015_2022-05-01', '9015_2022-06-01',
                  '9015_2022-07-01', '9015_2022-08-01', '9015_2022-09-01',
                  '9015_2022-10-01'],
, ordered=True)

In [159]:
# Turn categorical columns to numbers
for label, content in df_temp.items():
    if pd.api.types.is_categorical_dtype(content):
        # Turn into category codes starting with one
        df_temp[label] = pd.Categorical(content).codes+1

In [160]:
df_temp

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,month,year
0,118,1001,83,1,2019-08-01,3.007682,8,2019
41613,32410,21153,1027,18,2019-08-01,0.336289,8,2019
80652,72073,39057,685,36,2019-08-01,5.733927,8,2019
5460,103039,5063,811,4,2019-08-01,1.865025,8,2019
116805,111736,54021,643,49,2019-08-01,0.526540,8,2019
...,...,...,...,...,...,...,...,...
26675,17043,17181,1714,14,2022-10-01,1.637489,10,2022
95588,87594,47053,637,43,2022-10-01,2.517291,10,2022
43133,33930,21229,1770,18,2022-10-01,1.592288,10,2022
105143,97149,48353,1199,44,2022-10-01,1.502534,10,2022


In [161]:
# Remove first_day_of_month column as it is no longer useful
df_temp.drop('first_day_of_month', axis=1, inplace=True)

In [162]:
# Split into X & y
X = df_temp.drop('microbusiness_density', axis=1)
y = df_temp['microbusiness_density']
y

0         3.007682
41613     0.336289
80652     5.733927
5460      1.865025
116805    0.526540
            ...   
26675     1.637489
95588     2.517291
43133     1.592288
105143    1.502534
122264    1.785395
Name: microbusiness_density, Length: 122265, dtype: float64

In [163]:
X

,row_id,cfips,county,state,month,year
0,118,1001,83,1,8,2019
41613,32410,21153,1027,18,8,2019
80652,72073,39057,685,36,8,2019
5460,103039,5063,811,4,8,2019
116805,111736,54021,643,49,8,2019
...,...,...,...,...,...,...
26675,17043,17181,1714,14,10,2022
95588,87594,47053,637,43,10,2022
43133,33930,21229,1770,18,10,2022
105143,97149,48353,1199,44,10,2022


In [164]:
# Split data
train_size = 0.8
val_size = 0.2

X_train = X[:round(train_size*len(X))]
y_train = y[:round(train_size*len(y))]

X_val = X[len(X_train):]
y_val = y[len(y_train):]

In [165]:
X_train

,row_id,cfips,county,state,month,year
0,118,1001,83,1,8,2019
41613,32410,21153,1027,18,8,2019
80652,72073,39057,685,36,8,2019
5460,103039,5063,811,4,8,2019
116805,111736,54021,643,49,8,2019
...,...,...,...,...,...,...
116095,111026,53061,1550,48,3,2022
104434,96440,48317,1049,44,3,2022
79864,71285,39015,214,36,3,2022
58297,49718,29027,251,26,3,2022


In [166]:
y_train

0         3.007682
41613     0.336289
80652     5.733927
5460      1.865025
116805    0.526540
            ...   
116095    8.121901
104434    2.377875
79864     1.512580
58297     2.180545
99247     6.266245
Name: microbusiness_density, Length: 97812, dtype: float64

In [167]:
X_val

,row_id,cfips,county,state,month,year
95425,87431,47045,521,43,3,2022
67618,59039,31165,1541,28,3,2022
74794,66215,37061,517,34,3,2022
49717,41138,26091,972,23,3,2022
10873,120893,8071,952,6,3,2022
...,...,...,...,...,...,...
26675,17043,17181,1714,14,10,2022
95588,87594,47053,637,43,10,2022
43133,33930,21229,1770,18,10,2022
105143,97149,48353,1199,44,10,2022


In [168]:
y_val

95425     1.420915
67618     0.506586
74794     1.058036
49717     4.448828
10873     3.785031
            ...   
26675     1.637489
95588     2.517291
43133     1.592288
105143    1.502534
122264    1.785395
Name: microbusiness_density, Length: 24453, dtype: float64

In [169]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122265 entries, 0 to 122264
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   row_id                 122265 non-null  int32  
 1   cfips                  122265 non-null  int64  
 2   county                 122265 non-null  int16  
 3   state                  122265 non-null  int8   
 4   microbusiness_density  122265 non-null  float64
 5   month                  122265 non-null  int64  
 6   year                   122265 non-null  int64  
dtypes: float64(1), int16(1), int32(1), int64(3), int8(1)
memory usage: 5.5 MB


In [170]:
from sklearn.ensemble import RandomForestRegressor

In [171]:
import pandas as pd 
from pandas import MultiIndex, Int16Dtype
import xgboost as xgb

In [172]:
from xgboost import XGBRFRegressor

In [173]:
np.random.seed(42)

# Instantiate model
model = XGBRFRegressor(n_jobs=-1)

# FIt model
model.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBRFRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bytree=1, enable_categorical=False, gamma=0, gpu_id=-1,
               importance_type=None, interaction_constraints='',
               max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
               monotone_constraints='()', n_estimators=100, n_jobs=-1,
               num_parallel_tree=100, objective='reg:squarederror',
               predictor='auto', random_state=0, reg_alpha=0,
               scale_pos_weight=1, tree_method='exact', validate_parameters=1,
               verbosity=None)

In [174]:
model.score(X_train, y_train)

0.440226602773344

In [175]:
y_preds = model.predict(X_val)
y_preds

array([3.3695235, 2.9281871, 3.683925 , ..., 2.382376 , 3.3695235,
       2.1094105], dtype=float32)

In [176]:
from sklearn.metrics import mean_squared_error, mean_absolute_error 

In [177]:
mse = mean_squared_error(y_val, y_preds)
mse

20.107812168561345

In [178]:
mae = mean_absolute_error(y_val, y_preds)
mae

2.1147276327072477

In [179]:
def smape(y_val, y_preds):
    return 100/len(y_val) * np.sum(2 * np.abs(y_preds - y_val) / (np.abs(y_val) + np.abs(y_preds)))


print(smape(y_val, y_preds))

52.80221583623753


In [180]:
df_temp.corr()

,row_id,cfips,county,state,microbusiness_density,month,year
row_id,1.000000,6.391735e-01,6.130292e-02,6.464134e-01,0.084429,-1.371349e-05,3.056675e-04
cfips,0.639173,1.000000e+00,1.349395e-02,9.994320e-01,-0.011767,-1.595760e-15,-2.249911e-14
county,0.061303,1.349395e-02,1.000000e+00,9.634847e-03,0.015986,-6.518271e-16,-1.129626e-14
state,0.646413,9.994320e-01,9.634847e-03,1.000000e+00,-0.011107,-2.455960e-16,-2.057962e-14
microbusiness_density,0.084429,-1.176709e-02,1.598633e-02,-1.110732e-02,1.000000,2.121343e-03,1.702471e-02
month,-0.000014,-1.595760e-15,-6.518271e-16,-2.455960e-16,0.002121,1.000000e+00,-3.257756e-01
year,0.000306,-2.249911e-14,-1.129626e-14,-2.057962e-14,0.017025,-3.257756e-01,1.000000e+00


In [181]:
df_test = pd.read_csv('C:/Users\Boma Allwell-Brown/Downloads/godaddy-microbusiness-density-forecasting/test.csv',
                      parse_dates=['first_day_of_month'])
df_test

,row_id,cfips,first_day_of_month
0,1001_2022-11-01,1001,2022-11-01
1,1003_2022-11-01,1003,2022-11-01
2,1005_2022-11-01,1005,2022-11-01
3,1007_2022-11-01,1007,2022-11-01
4,1009_2022-11-01,1009,2022-11-01
...,...,...,...
25075,56037_2023-06-01,56037,2023-06-01
25076,56039_2023-06-01,56039,2023-06-01
25077,56041_2023-06-01,56041,2023-06-01
25078,56043_2023-06-01,56043,2023-06-01


In [182]:
df_temp

,row_id,cfips,county,state,microbusiness_density,month,year
0,118,1001,83,1,3.007682,8,2019
41613,32410,21153,1027,18,0.336289,8,2019
80652,72073,39057,685,36,5.733927,8,2019
5460,103039,5063,811,4,1.865025,8,2019
116805,111736,54021,643,49,0.526540,8,2019
...,...,...,...,...,...,...,...
26675,17043,17181,1714,14,1.637489,10,2022
95588,87594,47053,637,43,2.517291,10,2022
43133,33930,21229,1770,18,1.592288,10,2022
105143,97149,48353,1199,44,1.502534,10,2022


In [183]:
# Find missing columns not in df_test
missing_columns = set(df_temp.columns) - set(df_test)
missing_columns

{'county', 'microbusiness_density', 'month', 'state', 'year'}

In [184]:
df_microbusiness['county'].unique()

array(['Autauga County', 'Baldwin County', 'Barbour County', ...,
       'Uinta County', 'Washakie County', 'Weston County'], dtype=object)

In [186]:
df_test_copy = df_test.copy()
df_test_copy

,row_id,cfips,first_day_of_month
0,1001_2022-11-01,1001,2022-11-01
1,1003_2022-11-01,1003,2022-11-01
2,1005_2022-11-01,1005,2022-11-01
3,1007_2022-11-01,1007,2022-11-01
4,1009_2022-11-01,1009,2022-11-01
...,...,...,...
25075,56037_2023-06-01,56037,2023-06-01
25076,56039_2023-06-01,56039,2023-06-01
25077,56041_2023-06-01,56041,2023-06-01
25078,56043_2023-06-01,56043,2023-06-01


In [187]:
# Add column for month and year
df_test['month'] = df_test.first_day_of_month.dt.month
df_test['year'] = df_test.first_day_of_month.dt.year
df_test

,row_id,cfips,first_day_of_month,month,year
0,1001_2022-11-01,1001,2022-11-01,11,2022
1,1003_2022-11-01,1003,2022-11-01,11,2022
2,1005_2022-11-01,1005,2022-11-01,11,2022
3,1007_2022-11-01,1007,2022-11-01,11,2022
4,1009_2022-11-01,1009,2022-11-01,11,2022
...,...,...,...,...,...
25075,56037_2023-06-01,56037,2023-06-01,6,2023
25076,56039_2023-06-01,56039,2023-06-01,6,2023
25077,56041_2023-06-01,56041,2023-06-01,6,2023
25078,56043_2023-06-01,56043,2023-06-01,6,2023


In [188]:
# Find missing columns not in df_test
missing_columns = set(df_temp.columns) - set(df_test)
missing_columns

{'county', 'microbusiness_density', 'state'}

In [189]:
df_microbusiness

,row_id,cfips,county,state,first_day_of_month,microbusiness_density
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233
...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395


In [190]:
# Create dictionary for cfips: county
df_county_dict = pd.Series(df_microbusiness.county.values,index=df_microbusiness.cfips).to_dict()

In [191]:
df_county_dict

{1001: 'Autauga County',
 1003: 'Baldwin County',
 1005: 'Barbour County',
 1007: 'Bibb County',
 1009: 'Blount County',
 1011: 'Bullock County',
 1013: 'Butler County',
 1015: 'Calhoun County',
 1017: 'Chambers County',
 1019: 'Cherokee County',
 1021: 'Chilton County',
 1023: 'Choctaw County',
 1025: 'Clarke County',
 1027: 'Clay County',
 1029: 'Cleburne County',
 1031: 'Coffee County',
 1033: 'Colbert County',
 1035: 'Conecuh County',
 1037: 'Coosa County',
 1039: 'Covington County',
 1041: 'Crenshaw County',
 1043: 'Cullman County',
 1045: 'Dale County',
 1047: 'Dallas County',
 1049: 'DeKalb County',
 1051: 'Elmore County',
 1053: 'Escambia County',
 1055: 'Etowah County',
 1057: 'Fayette County',
 1059: 'Franklin County',
 1061: 'Geneva County',
 1063: 'Greene County',
 1065: 'Hale County',
 1067: 'Henry County',
 1069: 'Houston County',
 1071: 'Jackson County',
 1073: 'Jefferson County',
 1075: 'Lamar County',
 1077: 'Lauderdale County',
 1079: 'Lawrence County',
 1081: 'Lee Co

In [192]:
df_test

,row_id,cfips,first_day_of_month,month,year
0,1001_2022-11-01,1001,2022-11-01,11,2022
1,1003_2022-11-01,1003,2022-11-01,11,2022
2,1005_2022-11-01,1005,2022-11-01,11,2022
3,1007_2022-11-01,1007,2022-11-01,11,2022
4,1009_2022-11-01,1009,2022-11-01,11,2022
...,...,...,...,...,...
25075,56037_2023-06-01,56037,2023-06-01,6,2023
25076,56039_2023-06-01,56039,2023-06-01,6,2023
25077,56041_2023-06-01,56041,2023-06-01,6,2023
25078,56043_2023-06-01,56043,2023-06-01,6,2023


In [193]:
# Create county column in df_test
df_test['county'] = df_test['cfips'].map(df_county_dict)

In [194]:
df_test

,row_id,cfips,first_day_of_month,month,year,county
0,1001_2022-11-01,1001,2022-11-01,11,2022,Autauga County
1,1003_2022-11-01,1003,2022-11-01,11,2022,Baldwin County
2,1005_2022-11-01,1005,2022-11-01,11,2022,Barbour County
3,1007_2022-11-01,1007,2022-11-01,11,2022,Bibb County
4,1009_2022-11-01,1009,2022-11-01,11,2022,Blount County
...,...,...,...,...,...,...
25075,56037_2023-06-01,56037,2023-06-01,6,2023,Sweetwater County
25076,56039_2023-06-01,56039,2023-06-01,6,2023,Teton County
25077,56041_2023-06-01,56041,2023-06-01,6,2023,Uinta County
25078,56043_2023-06-01,56043,2023-06-01,6,2023,Washakie County


In [195]:
# Create dictionary for cfips: state 
df_state_dict = pd.Series(df_microbusiness.state.values,index=df_microbusiness.cfips).to_dict()
df_state_dict

{1001: 'Alabama',
 1003: 'Alabama',
 1005: 'Alabama',
 1007: 'Alabama',
 1009: 'Alabama',
 1011: 'Alabama',
 1013: 'Alabama',
 1015: 'Alabama',
 1017: 'Alabama',
 1019: 'Alabama',
 1021: 'Alabama',
 1023: 'Alabama',
 1025: 'Alabama',
 1027: 'Alabama',
 1029: 'Alabama',
 1031: 'Alabama',
 1033: 'Alabama',
 1035: 'Alabama',
 1037: 'Alabama',
 1039: 'Alabama',
 1041: 'Alabama',
 1043: 'Alabama',
 1045: 'Alabama',
 1047: 'Alabama',
 1049: 'Alabama',
 1051: 'Alabama',
 1053: 'Alabama',
 1055: 'Alabama',
 1057: 'Alabama',
 1059: 'Alabama',
 1061: 'Alabama',
 1063: 'Alabama',
 1065: 'Alabama',
 1067: 'Alabama',
 1069: 'Alabama',
 1071: 'Alabama',
 1073: 'Alabama',
 1075: 'Alabama',
 1077: 'Alabama',
 1079: 'Alabama',
 1081: 'Alabama',
 1083: 'Alabama',
 1085: 'Alabama',
 1087: 'Alabama',
 1089: 'Alabama',
 1091: 'Alabama',
 1093: 'Alabama',
 1095: 'Alabama',
 1097: 'Alabama',
 1099: 'Alabama',
 1101: 'Alabama',
 1103: 'Alabama',
 1105: 'Alabama',
 1107: 'Alabama',
 1109: 'Alabama',
 1111: 'Al

In [196]:
# Add state column to df_test based on dictionary (df_state_dict)
df_test['state'] = df_test['cfips'].map(df_state_dict)
df_test

,row_id,cfips,first_day_of_month,month,year,county,state
0,1001_2022-11-01,1001,2022-11-01,11,2022,Autauga County,Alabama
1,1003_2022-11-01,1003,2022-11-01,11,2022,Baldwin County,Alabama
2,1005_2022-11-01,1005,2022-11-01,11,2022,Barbour County,Alabama
3,1007_2022-11-01,1007,2022-11-01,11,2022,Bibb County,Alabama
4,1009_2022-11-01,1009,2022-11-01,11,2022,Blount County,Alabama
...,...,...,...,...,...,...,...
25075,56037_2023-06-01,56037,2023-06-01,6,2023,Sweetwater County,Wyoming
25076,56039_2023-06-01,56039,2023-06-01,6,2023,Teton County,Wyoming
25077,56041_2023-06-01,56041,2023-06-01,6,2023,Uinta County,Wyoming
25078,56043_2023-06-01,56043,2023-06-01,6,2023,Washakie County,Wyoming


In [197]:
# Convert string columns to category
for label, content in df_test.items():
    if pd.api.types.is_string_dtype(content):
        df_test[label] = content.astype('category').cat.as_ordered()

In [198]:
df_test.dtypes

row_id                      category
cfips                          int64
first_day_of_month    datetime64[ns]
month                          int64
year                           int64
county                      category
state                       category
dtype: object

In [199]:
df_test_copy

,row_id,cfips,first_day_of_month
0,1001_2022-11-01,1001,2022-11-01
1,1003_2022-11-01,1003,2022-11-01
2,1005_2022-11-01,1005,2022-11-01
3,1007_2022-11-01,1007,2022-11-01
4,1009_2022-11-01,1009,2022-11-01
...,...,...,...
25075,56037_2023-06-01,56037,2023-06-01
25076,56039_2023-06-01,56039,2023-06-01
25077,56041_2023-06-01,56041,2023-06-01
25078,56043_2023-06-01,56043,2023-06-01


In [200]:
# Turn categorical columns to numbers
for label, content in df_test.items():
    if pd.api.types.is_categorical_dtype(content):
        # Turn into category codes starting with one
        df_test[label] = pd.Categorical(content).codes+1

In [201]:
df_test_copy

,row_id,cfips,first_day_of_month
0,1001_2022-11-01,1001,2022-11-01
1,1003_2022-11-01,1003,2022-11-01
2,1005_2022-11-01,1005,2022-11-01
3,1007_2022-11-01,1007,2022-11-01
4,1009_2022-11-01,1009,2022-11-01
...,...,...,...
25075,56037_2023-06-01,56037,2023-06-01
25076,56039_2023-06-01,56039,2023-06-01
25077,56041_2023-06-01,56041,2023-06-01
25078,56043_2023-06-01,56043,2023-06-01


In [202]:
df_test.drop('first_day_of_month', axis=1, inplace=True)
df_test

,row_id,cfips,month,year,county,state
0,25,1001,11,2022,83,1
1,33,1003,11,2022,90,1
2,41,1005,11,2022,101,1
3,49,1007,11,2022,150,1
4,57,1009,11,2022,165,1
...,...,...,...,...,...,...
25075,24008,56037,6,2023,1631,51
25076,24016,56039,6,2023,1656,51
25077,24024,56041,6,2023,1709,51
25078,24032,56043,6,2023,1768,51


In [203]:
# Reorder columns as they were in training fit
df_test = df_test[X_train.columns]
df_test

,row_id,cfips,county,state,month,year
0,25,1001,83,1,11,2022
1,33,1003,90,1,11,2022
2,41,1005,101,1,11,2022
3,49,1007,150,1,11,2022
4,57,1009,165,1,11,2022
...,...,...,...,...,...,...
25075,24008,56037,1631,51,6,2023
25076,24016,56039,1656,51,6,2023
25077,24024,56041,1709,51,6,2023
25078,24032,56043,1768,51,6,2023


In [204]:
# Predict microbusiness_density
test_preds = pd.DataFrame(model.predict(df_test))
test_preds

,0
0,5.080000
1,5.520839
2,9.384253
3,9.437347
4,9.735255
...,...
25075,11.713676
25076,11.927357
25077,11.695789
25078,11.695789


In [205]:
df_preds = pd.DataFrame()
df_preds['row_id'] = df_test_copy['row_id']
df_preds['microbusiness_density'] = test_preds
df_preds

,row_id,microbusiness_density
0,1001_2022-11-01,5.080000
1,1003_2022-11-01,5.520839
2,1005_2022-11-01,9.384253
3,1007_2022-11-01,9.437347
4,1009_2022-11-01,9.735255
...,...,...
25075,56037_2023-06-01,11.713676
25076,56039_2023-06-01,11.927357
25077,56041_2023-06-01,11.695789
25078,56043_2023-06-01,11.695789


In [206]:
submission = df_preds.to_csv('godaddy_microbusiness density_forecast_submission.csv', index=False)

In [207]:
df_preds

,row_id,microbusiness_density
0,1001_2022-11-01,5.080000
1,1003_2022-11-01,5.520839
2,1005_2022-11-01,9.384253
3,1007_2022-11-01,9.437347
4,1009_2022-11-01,9.735255
...,...,...
25075,56037_2023-06-01,11.713676
25076,56039_2023-06-01,11.927357
25077,56041_2023-06-01,11.695789
25078,56043_2023-06-01,11.695789


In [208]:
df_test_copy

,row_id,cfips,first_day_of_month
0,1001_2022-11-01,1001,2022-11-01
1,1003_2022-11-01,1003,2022-11-01
2,1005_2022-11-01,1005,2022-11-01
3,1007_2022-11-01,1007,2022-11-01
4,1009_2022-11-01,1009,2022-11-01
...,...,...,...
25075,56037_2023-06-01,56037,2023-06-01
25076,56039_2023-06-01,56039,2023-06-01
25077,56041_2023-06-01,56041,2023-06-01
25078,56043_2023-06-01,56043,2023-06-01
